In [3]:
import torch
import numpy as np
import pickle
U1 = torch.load('/home/users/lewis841214/Desktop/1project/CryoEMCode/GPU_CryoEM/tomosipo_structureed/snap_shot/20220309_3DVA_data_mean_by_EM_PCA_after_subtract_senior_mean_EM_2SDR_n_20_18_is_pretrained_1th_4_U1.pt')
U2 = torch.load('/home/users/lewis841214/Desktop/1project/CryoEMCode/GPU_CryoEM/tomosipo_structureed/snap_shot/20220309_3DVA_data_mean_by_EM_PCA_after_subtract_senior_mean_EM_2SDR_n_20_18_is_pretrained_1th_4_U2.pt')
U3 = torch.load('/home/users/lewis841214/Desktop/1project/CryoEMCode/GPU_CryoEM/tomosipo_structureed/snap_shot/20220309_3DVA_data_mean_by_EM_PCA_after_subtract_senior_mean_EM_2SDR_n_20_18_is_pretrained_1th_4_U3.pt')

pca_file= '/home/users/lewis841214/Desktop/1project/CryoEMCode/GPU_CryoEM/tomosipo_structureed/snap_shot/20220309_3DVA_data_mean_by_EM_PCA_after_subtract_senior_mean_EM_2SDR_n_20_18_is_pretrained_PCA_W_iter_14.pkl'
with open(pca_file, 'rb') as f:
    PCA_coef = pickle.load(f)

In [45]:
import numpy
import mrcfile
ProjSize = 34
def save_density(data, grid_spacing, outfilename, origin=None):
    """
    Save the density to an mrc file. The origin of the grid will be (0,0,0)
    • outfilename: the mrc file name for the output
    """
    print("Saving mrc file ...") 
    data = data.astype('float32')
    with mrcfile.new(outfilename, overwrite=True) as mrc:
        mrc.set_data(data.T)
        mrc.voxel_size = grid_spacing
        if origin is not None:
            mrc.header['origin']['x'] = origin[0]
            mrc.header['origin']['y'] = origin[1]
            mrc.header['origin']['z'] = origin[2]
        mrc.update_header_from_data()
        mrc.update_header_stats()
    print("done") 
def reshape_fortran(x, shape):
    if len(x.shape) > 0:
        x = x.permute(*reversed(range(len(x.shape))))
    return x.reshape(*reversed(shape)).permute(*reversed(range(len(shape))))
All_U= torch.kron(U1, torch.kron(U2, U3))
PCs = torch.permute(torch.tensor(PCA_coef), (1, 0)).float()
two_SDR_PCs = All_U @ PCs
two_SDR_PCs_struct = reshape_fortran(two_SDR_PCs, (5, ProjSize, ProjSize, ProjSize)).float().detach().numpy()

In [46]:
print(two_SDR_PCs_struct.shape)
two_SDR_PCs_struct = torch.load('/home/users/lewis841214/Desktop/1project/CryoEMCode/GPU_CryoEM/tomosipo_structureed/snap_shot/20220309_3DVA_data_mean_by_EM_PCA_after_subtract_senior_mean_EM_2SDR_n_20_18_is_pretrained_lr0.01_1th_2SDR_PCs.pt')
two_SDR_PCs_struct= reshape_fortran(two_SDR_PCs_struct, (5, ProjSize, ProjSize, ProjSize)).float().detach().numpy()
first_struct = two_SDR_PCs_struct[2]
save_density(first_struct, (68,68,68), 'test.mrc' )
import nglview as nv
view_density_map = nv.show_file(f"test.mrc")
view_density_map

(5, 34, 34, 34)
Saving mrc file ...
done


NGLWidget()

In [34]:
print(first_struct.shape)
print(type(first_struct.shape))
mu_file= '/home/users/lewis841214/Desktop/1project/CryoEMCode/GPU_CryoEM/tomosipo_structureed/snap_shot/20220309_3DVA_data_mean_by_EM_PCA_after_subtract_senior_mean_EM_2SDR_n_20_18_is_pretrained_all_mu_14.pkl'
with open(pca_file, 'rb') as f:
    all_mu = pickle.load(f)
print(all_mu.shape)

(34, 34, 34)
<class 'tuple'>
(5, 8000)


In [42]:
print(two_SDR_PCs_struct.shape)
for i in range(5):
    if i ==0:
        f_struct = two_SDR_PCs_struct[0] * all_mu[:,0][0]
    else:
        f_struct = f_struct + two_SDR_PCs_struct[i] * all_mu[:,0][i]
        
print(f_struct.shape)
save_density(f_struct, (68,68,68), 'test.mrc' )
import nglview as nv
view_density_map = nv.show_file(f"test.mrc")
view_density_map

(5, 34, 34, 34)
(34, 34, 34)
Saving mrc file ...
done


NGLWidget()

In [59]:
mu_file= '/home/users/lewis841214/Desktop/1project/CryoEMCode/GPU_CryoEM/tomosipo_structureed/snap_shot/20220309_3DVA_data_mean_by_EM_PCA_after_subtract_senior_mean_EM_2SDR_n_20_18_is_pretrained_all_mu_14.pkl'
with open(pca_file, 'rb') as f:
    all_mu = pickle.load(f)
print(type(all_mu))
print(torch.tensor(all_mu).float().T.shape)
sll_struct = All_U @ torch.tensor(all_mu).float().T
print(sll_struct.shape)
f_struct = sll_struct[:, 0]
f_struct = reshape_fortran(f_struct, (ProjSize, ProjSize, ProjSize)).float().detach().numpy()
save_density(f_struct, (68,68,68), 'test.mrc' )
import nglview as nv
view_density_map = nv.show_file(f"test.mrc")
view_density_map

<class 'numpy.ndarray'>
torch.Size([8000, 5])
torch.Size([39304, 5])
Saving mrc file ...
done


NGLWidget()

In [65]:
print(PCs.shape)
pc_struct = All_U @ torch.tensor(PCs).float()
f_pc_struct = pc_struct[:, 0]
f_pc_struct = reshape_fortran(f_pc_struct, (ProjSize, ProjSize, ProjSize)).float().detach().numpy()
save_density(f_pc_struct, (68,68,68), 'test.mrc' )
import nglview as nv
view_density_map = nv.show_file(f"test.mrc")
view_density_map

torch.Size([8000, 5])
Saving mrc file ...
done


/home/users/lewis841214/.conda/envs/protein_reconstruction/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


NGLWidget()